In [20]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, MaxPooling1D, Flatten, Embedding, Input
from keras.layers.convolutional import Conv1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping
from tensorboard.plugins.pr_curve import summary as pr_summary
import matplotlib.pyplot as plt

%matplotlib inline

# Data Preprocessing

In [32]:
Xs_data = ['neuron_params_d_I_samples_220K.csv', 'neuron_params_I_d_a_b_c_samples_110K.csv', 'neuron_params_I_d_a_b_samples_110K.csv', 'neuron_params_I_d_a_samples_110K.csv', 'neuron_params_I_samples_110K.csv']
ys_data = ['neuron_params_d_I_labels_220K.csv', 'neuron_params_I_d_a_b_c_labels_110K.csv', 'neuron_params_I_d_a_b_labels_110K.csv', 'neuron_params_I_d_a_labels_110K.csv', 'neuron_params_I_labels_110K.csv']
Xs = []
ys = []

In [33]:
for path in Xs_data:
    Xs.append(pd.read_csv('../../datasets/parametric_neuron_data/' + path, index_col=0, header=None).T)
    
for path in ys_data:
    ys.append(pd.read_csv('../../datasets/parametric_neuron_data/' + path, index_col=0, header=None).T)

In [87]:
# Add "header" as data
for X in Xs:
    X.loc[-1] = X.columns
    
for y in ys:
    y.loc[-1] = y.columns
    
# Select dataset
dataset = 1
X = Xs[dataset]
y = ys[dataset]
(num_samples, num_features) = X.shape

# Normalize input
X = X.apply(zscore)

# 80/20 train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

# Add an extra dimension to  XX  for 1-D convolution.
X_train_cnn = np.expand_dims(X_train.as_matrix(), axis=2)
X_test_cnn = np.expand_dims(X_test.as_matrix(), axis=2)

# TensorBoard Visualization

Displays Precision-Recall curve for Keras models.

Adapted from: https://medium.com/@akionakas/precision-recall-curve-with-keras-cd92647685e1

In [35]:
class PRTensorBoard(TensorBoard):
    def __init__(self, *args, **kwargs):
        # One extra argument to indicate whether or not to use the PR curve summary.
        self.pr_curve = kwargs.pop('pr_curve', True)
        super(PRTensorBoard, self).__init__(*args, **kwargs)

        global tf
        import tensorflow as tf

    def set_model(self, model):
        super(PRTensorBoard, self).set_model(model)

        if self.pr_curve:
            # Get the prediction and label tensor placeholders.
            predictions = self.model._feed_outputs[0]
            labels = tf.cast(self.model._feed_targets[0], tf.bool)
            # Create the PR summary OP.
            self.pr_summary = pr_summary.op(name='pr_curve',
                                            predictions=predictions,
                                            labels=labels,
                                            display_name='Precision-Recall Curve')

    def on_epoch_end(self, epoch, logs=None):
        super(PRTensorBoard, self).on_epoch_end(epoch, logs)

        if self.pr_curve and self.validation_data:
            # Get the tensors again.
            tensors = self.model._feed_targets + self.model._feed_outputs
            # Predict the output.
            predictions = self.model.predict(self.validation_data[0])
            # Build the dictionary mapping the tensor to the data.
            val_data = [self.validation_data[1], predictions]
            feed_dict = dict(zip(tensors, val_data))
            # Run and add summary.
            result = self.sess.run([self.pr_summary], feed_dict=feed_dict)
            self.writer.add_summary(result[0], epoch)
        self.writer.flush()

# CNN Model

In [36]:
epochs = 10
batch_size = 32

## Dataset 0

In [79]:
# 16 3x1 convolutions
inputs = Input(shape=(num_features, 1))
conv1 = Conv1D(4, 3, padding='same', strides=1)(inputs)
bn1 = BatchNormalization()(conv1)
relu1 = Activation('relu')(bn1)

# 32 3x16 convolutions
conv2 = Conv1D(8, 3, padding='same', strides=1)(relu1)
bn2 = BatchNormalization()(conv2)
relu2 = Activation('relu')(bn2)
flatten = Flatten()(relu2)

# FC 2048
fc1 = Dense(32)(flatten)
bn3 = BatchNormalization()(fc1)
relu3 = Activation('relu')(bn3)

# FC 1024
fc2 = Dense(16)(relu3)
bn4 = BatchNormalization()(fc2)
relu4 = Activation('relu')(bn4)

# FC 512
fc3 = Dense(16)(relu4)
bn5 = BatchNormalization()(fc3)
relu5 = Activation('relu')(bn5)

# Output
fc4 = Dense(1)(relu5)
outputs = Activation('sigmoid')(fc4)

model = Model(inputs=inputs, outputs=outputs)

In [84]:
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [85]:
model.fit(X_train_cnn, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[X_test_cnn, y_test], # NOTE: must pass test data for PRTensorBoard callback
          shuffle=True,
          callbacks=[PRTensorBoard(log_dir='logs/adam_dropout-50_dataset-' + str(dataset)), EarlyStopping(monitor='val_acc', patience=2)])

Train on 176000 samples, validate on 44000 samples
Epoch 1/10
176000/176000 [==============================] - 53s 302us/step - loss: 0.0380 - acc: 0.9839 - val_loss: 0.0335 - val_acc: 0.9865
Epoch 2/10
176000/176000 [==============================] - 39s 223us/step - loss: 0.0384 - acc: 0.9837 - val_loss: 0.0336 - val_acc: 0.9862
Epoch 3/10
176000/176000 [==============================] - 39s 223us/step - loss: 0.0380 - acc: 0.9839 - val_loss: 0.0339 - val_acc: 0.9862


## Dataset 1

In [88]:
###BEST 0.9721
# 16 3x1 convolutions
inputs = Input(shape=(num_features, 1))
conv1 = Conv1D(16, 3, padding='same', strides=1)(inputs)
bn1 = BatchNormalization()(conv1)
relu1 = Activation('relu')(bn1)

# 32 3x16 convolutions
conv2 = Conv1D(32, 3, padding='same', strides=1)(relu1)
bn2 = BatchNormalization()(conv2)
relu2 = Activation('relu')(bn2)
flatten = Flatten()(relu2)

# FC 2048
fc1 = Dense(256)(flatten)
bn3 = BatchNormalization()(fc1)
relu3 = Activation('relu')(bn3)
drop1 = Dropout(0.5)(relu3)

# FC 1024
fc2 = Dense(128)(drop1)
bn4 = BatchNormalization()(fc2)
relu4 = Activation('relu')(bn4)
drop2 = Dropout(0.5)(relu4)

# FC 512
fc3 = Dense(64)(drop2)
bn5 = BatchNormalization()(fc3)
relu5 = Activation('relu')(bn5)
drop3 = Dropout(0.5)(relu5)

# Output
fc4 = Dense(1)(drop3)
outputs = Activation('sigmoid')(fc4)

model = Model(inputs=inputs, outputs=outputs)

opt = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [101]:
##
## TEST
##
inputs = Input(shape=(num_features, 1))
conv1 = Conv1D(16, 3, padding='same', strides=1)(inputs)
bn1 = BatchNormalization()(conv1)
relu1 = Activation('relu')(bn1)

# 32 3x16 convolutions
conv2 = Conv1D(32, 3, padding='same', strides=1)(relu1)
bn2 = BatchNormalization()(conv2)
relu2 = Activation('relu')(bn2)
flatten = Flatten()(relu2)

# FC 2048
fc1 = Dense(128)(flatten)
bn3 = BatchNormalization()(fc1)
relu3 = Activation('relu')(bn3)
drop1 = Dropout(0.5)(relu3)

# FC 1024
fc2 = Dense(64)(drop1)
bn4 = BatchNormalization()(fc2)
relu4 = Activation('relu')(bn4)
drop2 = Dropout(0.5)(relu4)

# FC 512
fc3 = Dense(32)(drop2)
bn5 = BatchNormalization()(fc3)
relu5 = Activation('relu')(bn5)
drop3 = Dropout(0.5)(relu5)

# Output
fc4 = Dense(1)(drop3)
outputs = Activation('sigmoid')(fc4)

model = Model(inputs=inputs, outputs=outputs)

In [104]:
opt = keras.optimizers.Nadam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [105]:
model.fit(X_train_cnn, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[X_test_cnn, y_test], # NOTE: must pass test data for PRTensorBoard callback
          shuffle=True,
          callbacks=[PRTensorBoard(log_dir='logs/nadam_dropout-50_dataset-' + str(dataset)), EarlyStopping(monitor='val_acc', patience=2)])

Train on 88000 samples, validate on 22000 samples
Epoch 1/10
88000/88000 [==============================] - 30s 346us/step - loss: 0.1248 - acc: 0.9525 - val_loss: 0.0874 - val_acc: 0.9647
Epoch 2/10
88000/88000 [==============================] - 30s 335us/step - loss: 0.1163 - acc: 0.9561 - val_loss: 0.0866 - val_acc: 0.9654
Epoch 3/10
88000/88000 [==============================] - 25s 286us/step - loss: 0.1094 - acc: 0.9591 - val_loss: 0.0829 - val_acc: 0.9674
Epoch 4/10
88000/88000 [==============================] - 25s 285us/step - loss: 0.1105 - acc: 0.9587 - val_loss: 0.0793 - val_acc: 0.9695
Epoch 5/10
88000/88000 [==============================] - 25s 286us/step - loss: 0.1063 - acc: 0.9604 - val_loss: 0.0804 - val_acc: 0.9685
Epoch 6/10
88000/88000 [==============================] - 26s 298us/step - loss: 0.1039 - acc: 0.9606 - val_loss: 0.0782 - val_acc: 0.9705
Epoch 7/10
88000/88000 [==============================] - 25s 288us/step - loss: 0.1022 - acc: 0.9614 - val_loss: 0.

# Performance Scores

In [ ]:
results = [0.98625, 0.9721, 0, 0, 0]